In [1]:
import gymnasium as gym
import numpy as np

In [2]:
gym.pprint_registry()

===== classic_control =====
Acrobot-v1             CartPole-v0            CartPole-v1
MountainCar-v0         MountainCarContinuous-v0 Pendulum-v1
===== phys2d =====
phys2d/CartPole-v0     phys2d/CartPole-v1     phys2d/Pendulum-v0
===== box2d =====
BipedalWalker-v3       BipedalWalkerHardcore-v3 CarRacing-v3
LunarLander-v3         LunarLanderContinuous-v3
===== toy_text =====
Blackjack-v1           CliffWalking-v0        FrozenLake-v1
FrozenLake8x8-v1       Taxi-v3
===== tabular =====
tabular/Blackjack-v0   tabular/CliffWalking-v0
===== mujoco =====
Ant-v2                 Ant-v3                 Ant-v4
Ant-v5                 HalfCheetah-v2         HalfCheetah-v3
HalfCheetah-v4         HalfCheetah-v5         Hopper-v2
Hopper-v3              Hopper-v4              Hopper-v5
Humanoid-v2            Humanoid-v3            Humanoid-v4
Humanoid-v5            HumanoidStandup-v2     HumanoidStandup-v4
HumanoidStandup-v5     InvertedDoublePendulum-v2 InvertedDoublePendulum-v4
InvertedDoublePendulu

In [3]:
env = gym.make('CliffWalking-v0', render_mode='human')

In [9]:
class QLearningAgent:
    
    def __init__(
        self,
        environ, 
        n_episodes,
        epsilon=0.6,
        learning_rate=0.9,
        discount_factor=0.95,
        random=False,
        max_epsilon=0.1
        ) -> None:
        
        self.env = environ
        self.nr_of_actions = environ.action_space.n
        self.nr_of_states = environ.observation_space.n
        
        self.epsilon = epsilon
        self.epsilon_decay = self.epsilon / (n_episodes / 2)
        self.max_epsilon = max_epsilon
        self.lr = learning_rate
        self.gamma = discount_factor
        
        # in this case [0, 1] indicates state 0 with action 1
        if random:
            self.q_function = np.random.rand(self.nr_of_states, self.nr_of_actions)
        else:
            self.q_function = np.zeros((self.nr_of_states, self.nr_of_actions))
        
    def update(
        self,
        state,
        action,
        reward, 
        terminated,
        next_state
        ) -> None:
        # get the max_a q(s, a)
        greedy_action = np.max(self.q_function[next_state]) if not terminated else 0
        self.q_function[state, action] += self.lr * (reward + self.gamma * greedy_action - self.q_function[state, action])
    
    def epsilon_greedy(
        self,
        state
    ) -> np.int64:
        if np.random.random() < self.epsilon:
            action = self.env.action_space.sample()
        else:
            action = int(np.argmax(self.q_function[state]))
        return action 
    
    def decay_epsilon(self) -> None:
        self.epsilon = max(self.max_epsilon, self.epsilon - self.epsilon_decay)
        
    def predict(
        self,
        state
        ) -> np.int64:
        return int(np.argmax(self.q_function[state]))

In [10]:
from tqdm import tqdm

n_eps = 100
agent = QLearningAgent(environ=env, n_episodes=n_eps)

env = gym.wrappers.RecordEpisodeStatistics(env=env, buffer_length=n_eps)

for episode in tqdm(range(n_eps)):
    curr_state, info = env.reset()
    done = False
    
    print(f'Episode: {episode+1} | epsilon: {agent.epsilon}')
    
    while not done:
        action = agent.epsilon_greedy(curr_state)
        next_state, reward, terminated, truncated, info = env.step(action)
        
        agent.update(curr_state, action, reward, terminated, next_state)
        
        done = terminated or truncated
        curr_state = next_state
    
    agent.decay_epsilon()

  0%|          | 0/100 [00:00<?, ?it/s]

Episode: 1 | epsilon: 0.6


  1%|          | 1/100 [04:24<7:17:14, 265.00s/it]

Episode: 2 | epsilon: 0.588


  2%|▏         | 2/100 [05:09<3:41:01, 135.32s/it]

...

 98%|█████████▊| 98/100 [17:14<00:08,  4.17s/it]

Episode: 99 | epsilon: 0.1


 99%|█████████▉| 99/100 [17:18<00:04,  4.19s/it]

Episode: 100 | epsilon: 0.1


100%|██████████| 100/100 [17:25<00:00, 10.45s/it]


In [11]:
for iteration in range(15):
    curr_state, info = env.reset()
    done = False
    
    reward_total = 0
    while not done:
        action = agent.predict(curr_state)
        next_state, reward, terminated, truncated, info = env.step(action)
        
        reward_total += reward
        done = terminated or truncated
        curr_state = next_state
        
    print(f'Episode: {iteration+1} | total reward at this episode: {reward_total}')

Episode: 1 | total reward at this episode: -13
Episode: 2 | total reward at this episode: -13
Episode: 3 | total reward at this episode: -13
Episode: 4 | total reward at this episode: -13
Episode: 5 | total reward at this episode: -13
Episode: 6 | total reward at this episode: -13
Episode: 7 | total reward at this episode: -13
Episode: 8 | total reward at this episode: -13
Episode: 9 | total reward at this episode: -13
Episode: 10 | total reward at this episode: -13
Episode: 11 | total reward at this episode: -13
Episode: 12 | total reward at this episode: -13
Episode: 13 | total reward at this episode: -13
Episode: 14 | total reward at this episode: -13
Episode: 15 | total reward at this episode: -13
